<a href="https://colab.research.google.com/github/abyanrifqy/ProjectSingkat/blob/master/flask-colab/livingroom_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! git clone https://github.com/droxey/ml-xception.git

Cloning into 'ml-xception'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 32 (delta 0), reused 0 (delta 0), pack-reused 29
Unpacking objects: 100% (32/32), done.


In [ ]:
!pip install flask-ngrok

In [ ]:
import os
import io
import numpy as np
import json

import keras
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from keras.applications.xception import preprocess_input
from tensorflow import keras
from keras import backend as K

from flask_ngrok import run_with_ngrok
from flask import Flask, request, redirect, url_for, jsonify

app = Flask(__name__)
run_with_ngrok(app)
# folder to storage, can be changed into cloud storage?
app.config['UPLOAD_FOLDER'] = '/content/Upload'

model = None
graph = None


def load_model():
    global model
    global graph
    model_path = "/content/drive/Shareddrives/Capstone_Project/Room Classification/model_cloud_tflite/cloud_livingroom"
    model = keras.models.load_model(model_path)
    graph = K.get_session().graph


load_model()


def prepare_image(img):
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    # return the processed image
    return img

names = ["poor","rich"]

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    data = {"success": False}
    if request.method == 'POST':
        if request.files.get('file'):
            file = request.files['file']
            filename = file.filename
            filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(filepath)

            # Load the saved image using Keras.
            # Resize it to the Xception format of 299x299 pixels.
            image_size = (299, 299)
            im = keras.preprocessing.image.load_img(filepath,
                                                    target_size=image_size,
                                                    grayscale=False)

            # Preprocess the image and prepare it for classification.
            image = prepare_image(im)

            global graph
            with graph.as_default():
                model_path = "/content/drive/Shareddrives/Capstone_Project/Room Classification/model_cloud_tflite/cloud_livingroom"
                model = keras.models.load_model(model_path)
                graph = K.get_session().graph
                preds = model.predict(image)
                print(preds)
                data["predictions"] = []

                # for (imagenetID, label, prob) in results[0]:
                #    r = {"label": label, "probability": float(prob)}
                #    data["predictions"].append(r)

                prob = preds
                if prob > 0.45:
                  labelName = names[1]
                else : 
                  labelName = names[0]
                  prob = 1-prob
                r = {"filename": filename, "result": labelName, "prob": float(prob)}
                data["predictions"].append(r)

                # indicate that the request was a success
                data["success"] = True

        return jsonify(data)

    return '''
    <!doctype html>
    <title>Upload new File</title>
    <h1>Upload new File</h1>
    <form method=post enctype=multipart/form-data>
      <p><input type=file name=file>
         <input type=submit value=Upload>
    </form>
    '''

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://2f7b0b161b71.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [09/Jun/2021 09:21:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2021 09:21:50] "GET /favicon.ico HTTP/1.1" 404 -


Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
127.0.0.1 - - [09/Jun/2021 09:22:27] "POST / HTTP/1.1" 200 -


[[0.00138953]]


In [ ]:
pip3 freeze > requirements.txt

SyntaxError: ignored